# Advanced Demo: Finance + News Assistant using OpenAI Tools and Simulated MCP 🧠📈📰

This notebook shows a **more complex, realistic use case** of:

- **Real OpenAI tool-calling** (`chat.completions.create` with `tools=`), and
- **Simulated MCP tools** that wrap:
  - Yahoo Finance (via `yfinance`), and
  - Tavily Search API (for company/news search).

We build a mini **"Portfolio Risk & News Assistant"** where the user can ask:

> *"I hold 50% AAPL and 50% MSFT. Give me current prices, 1-day change, 3 key recent news items per stock, and 2 major risks I should watch this quarter."*

The flow will be:

1. Define simulated **MCP tools**: `get_stock_snapshot`, `get_company_news`.
2. Convert these tools → **OpenAI tools schema**.
3. Call a **real OpenAI model** with `tools=`.
4. Let the model decide which tools to call and with what arguments.
5. Execute the tools in Python (simulating an MCP client calling an MCP server).
6. Call the model again with tool results to obtain a **final, well-structured answer**.

Every code cell is preceded by a detailed explanation so students can understand exactly what is happening.


## 0. Setup – Install Libraries and Configure API Keys

You will need:

- `OPENAI_API_KEY` – for OpenAI.
- `TAVILY_API_KEY` – for Tavily Search (https://tavily.com).

We also install:

- `openai` – official OpenAI Python client.
- `yfinance` – for Yahoo Finance data.
- `tavily-python` – Tavily client.

Run the cell below. In Colab, you may need to uncomment the `pip install` lines.


In [ ]:
# If running in Colab or a fresh environment, uncomment these:
# !pip install --upgrade openai yfinance tavily-python

import os, json, datetime
from typing import Callable, List, Dict, Any

import yfinance as yf
from openai import OpenAI
from tavily import TavilyClient

# Check environment variables
openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

print("OPENAI_API_KEY set:", bool(openai_key))
print("TAVILY_API_KEY set:", bool(tavily_key))

if not openai_key:
    print("⚠️ Please set OPENAI_API_KEY before running tool calls.")
if not tavily_key:
    print("⚠️ Please set TAVILY_API_KEY before running Tavily search.")

client = OpenAI(api_key=openai_key) if openai_key else None
tavily = TavilyClient(api_key=tavily_key) if tavily_key else None

# Model that supports tools
MODEL = "gpt-4o-mini"  # adjust if needed


## 1. Simulated MCP Server: Finance + News Tools

Imagine an MCP server that exposes these **capabilities**:

1. `get_stock_snapshot` – given a ticker, return:
   - current price
   - day change in %
   - 52-week high/low
2. `get_company_news` – given a ticker and optional time window, return
   - a short summary of recent news (via Tavily).

We simulate what `list_tools()` would return as `mcp_tools`.


In [ ]:
mcp_tools: List[Dict[str, Any]] = [
    {
        "name": "get_stock_snapshot",
        "description": "Get current price, daily change, and 52-week range for a stock ticker.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "Stock ticker symbol (e.g., AAPL, MSFT).",
                }
            },
            "required": ["ticker"],
        },
    },
    {
        "name": "get_company_news",
        "description": "Search recent web/news for a company (by ticker) and return key bullets.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "Stock ticker symbol.",
                },
                "days": {
                    "type": "integer",
                    "description": "How many recent days of news context to focus on.",
                    "default": 7,
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of search results to use.",
                    "default": 5,
                },
            },
            "required": ["ticker"],
        },
    },
]

print("Simulated MCP tools (like list_tools() output):")
for t in mcp_tools:
    print(f"- {t['name']}: {t['description']}")


## 2. MCP Client: Convert Tools → OpenAI `tools` Schema

Next, an MCP client translates these MCP tool descriptions into
OpenAI tool definitions that can be passed to the model.

The pattern is:

```python
tools = [
  {
    "type": "function",
    "function": {
      "name": ..., "description": ..., "parameters": ...
    }
  },
  ...
]
```

We implement a helper to do this conversion.


In [ ]:
def mcp_tools_to_openai_tools(mcp_tools: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    openai_tools: List[Dict[str, Any]] = []
    for tool in mcp_tools:
        openai_tools.append(
            {
                "type": "function",
                "function": {
                    "name": tool["name"],
                    "description": tool["description"],
                    "parameters": tool["inputSchema"],
                },
            }
        )
    return openai_tools

openai_tools = mcp_tools_to_openai_tools(mcp_tools)

print("OpenAI tools schema that will be sent to the model:\n")
print(json.dumps(openai_tools, indent=2))


## 3. Implement Tool Logic (Simulated MCP `call_tool`)

Now we define actual Python functions for our tools. In a real MCP server these
would be the bodies of `@mcp.tool()` functions.

### `get_stock_snapshot(ticker)`

Uses **Yahoo Finance** via `yfinance` to fetch:

- current price,
- day change %, and
- 52-week high/low.

### `get_company_news(ticker, days, max_results)`

Uses **Tavily** to search for recent news about the company.
We keep the output compact and structured so the model can reason about it.


In [ ]:
def tool_get_stock_snapshot(ticker: str) -> str:
    """Fetch basic stock snapshot using Yahoo Finance (yfinance).

    Returns a compact string summary for the LLM.
    """
    try:
        t = yf.Ticker(ticker)
        info = t.fast_info
        current = info.get("last_price") or info.get("last_close")
        prev_close = info.get("previous_close") or info.get("last_close")
        high_52w = info.get("year_high")
        low_52w = info.get("year_low")
        currency = info.get("currency", "USD")

        change_pct = None
        if current and prev_close and prev_close != 0:
            change_pct = (current - prev_close) / prev_close * 100.0

        parts = [f"Ticker: {ticker}"]
        if current is not None:
            parts.append(f"Current price: {current:.2f} {currency}")
        if change_pct is not None:
            parts.append(f"Day change: {change_pct:+.2f}% vs previous close")
        if low_52w is not None and high_52w is not None:
            parts.append(f"52-week range: {low_52w:.2f}–{high_52w:.2f} {currency}")

        return " | ".join(parts)
    except Exception as e:  # noqa: BLE001
        return f"Error fetching data for {ticker}: {e}"


def tool_get_company_news(ticker: str, days: int = 7, max_results: int = 5) -> str:
    """Use Tavily to fetch recent news-like context for the company.

    Returns a short bullet-style text summary.
    """
    if tavily is None:
        return "Tavily client not configured (no TAVILY_API_KEY)."

    today = datetime.date.today()
    start_date = today - datetime.timedelta(days=days)
    query = f"{ticker} stock company news after {start_date.isoformat()}"

    try:
        resp = tavily.search(
            query=query,
            search_depth="basic",
            max_results=max_results,
        )
    except Exception as e:  # noqa: BLE001
        return f"Error querying Tavily for {ticker}: {e}"

    results = resp.get("results", []) if isinstance(resp, dict) else resp
    if not results:
        return f"No recent results found for {ticker}."

    bullets = []
    for r in results:
        title = r.get("title", "(no title)")
        snippet = r.get("content", "")[:200]
        bullets.append(f"- {title}: {snippet}...")

    return "\n".join(bullets)


# Registry mapping tool names → implementations
TOOL_REGISTRY: Dict[str, Callable[..., str]] = {
    "get_stock_snapshot": tool_get_stock_snapshot,
    "get_company_news": tool_get_company_news,
}

print("Registered local tool implementations:")
for name in TOOL_REGISTRY:
    print("-", name)


## 4. Helper – Execute Tool Calls from the Model

When the OpenAI model responds with `tool_calls`, we need to:

1. Parse the **tool name** and **arguments JSON**.
2. Look up the function in `TOOL_REGISTRY`.
3. Run it.
4. Create a `role="tool"` message with the result, which will be fed back to the model.

In a real MCP setup, step 3 would actually call `client_session.call_tool(...)` and talk
to a separate MCP server. Here we keep it in-process for clarity.


In [ ]:
def execute_tool_call_from_model(tool_call) -> Dict[str, Any]:
    """Execute one tool_call object from the model and return a tool message.

    This simulates what an MCP client would do after reading tool_calls.
    """
    tool_name = tool_call.function.name
    args = json.loads(tool_call.function.arguments or "{}")
    fn = TOOL_REGISTRY.get(tool_name)
    if fn is None:
        content = f"Error: unknown tool '{tool_name}'"
    else:
        try:
            content = fn(**args)
        except Exception as e:  # noqa: BLE001
            content = f"Error executing tool {tool_name}: {e}"

    tool_message = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": content,
    }
    return tool_message


## 5. First OpenAI Call – Complex Portfolio Query

We now send a **richer user query** that involves multiple tickers and higher-level
reasoning:

> *"I hold 50% AAPL and 50% MSFT. Give me current prices, 1-day change, 2–3 key recent news items per stock, and 2 main risks I should watch this quarter."*

We include a **system message** gently instructing the model to use tools for
live finance/news data whenever possible.

The model will:

1. Read the tool list (`get_stock_snapshot`, `get_company_news`).
2. Decide which ones to call and with what arguments.
3. Return `tool_calls` describing those calls.


In [ ]:
if client is None:
    print("❌ OpenAI client is not configured. Set OPENAI_API_KEY and rerun setup.")
else:
    system_prompt = (
        "You are a portfolio risk assistant. "
        "Use tools for factual finance data and recent news when available. "
        "Then synthesize risks and insights in clear language."
    )

    user_query = (
        "I hold 50% AAPL and 50% MSFT. "
        "Give me current prices, 1-day change, 2–3 key recent news items per stock, "
        "and 2 main risks I should watch this quarter."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query},
    ]

    print("Sending first request to OpenAI with tools...\n")
    first_response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=openai_tools,
    )

    first_message = first_response.choices[0].message
    print("Raw first message from model (repr):\n")
    print(first_message)

    tool_calls = first_message.tool_calls or []
    if tool_calls:
        print("\n✅ Model requested tool calls:")
        for tc in tool_calls:
            print(f"- Tool name: {tc.function.name}")
            print(f"  Arguments JSON: {tc.function.arguments}")
    else:
        print("\nℹ️ Model did not request any tool calls.")


## 6. Execute Tool Calls and Ask for Final Answer

Now we:

1. Execute each tool call via `execute_tool_call_from_model(...)`.
2. Build the **full conversation history**:
   - system message,
   - user message,
   - first assistant message (with tool_calls),
   - all `role="tool"` messages with tool outputs.
3. Call the model again and let it synthesize a final answer that mixes:
   - numbers from Yahoo Finance,
   - recent news from Tavily,
   - and its own reasoning about portfolio risk.


In [ ]:
if client is None:
    print("❌ OpenAI client is not configured. Set OPENAI_API_KEY and rerun setup.")
else:
    tool_messages: List[Dict[str, Any]] = []
    for tc in (tool_calls or []):
        tm = execute_tool_call_from_model(tc)
        tool_messages.append(tm)

    print("Tool messages generated after executing tools:\n")
    print(json.dumps(tool_messages, indent=2))

    # Build full conversation history
    full_messages: List[Dict[str, Any]] = []
    full_messages.append({"role": "system", "content": system_prompt})
    full_messages.append({"role": "user", "content": user_query})

    if tool_calls:
        full_messages.append({
            "role": "assistant",
            "content": first_message.content or "",
            "tool_calls": [tc.to_dict() for tc in tool_calls],
        })
        full_messages.extend(tool_messages)
    else:
        full_messages.append({
            "role": "assistant",
            "content": first_message.content or "",
        })

    print("\nSending second request to OpenAI with tool results...\n")
    second_response = client.chat.completions.create(
        model=MODEL,
        messages=full_messages,
    )

    final_message = second_response.choices[0].message
    print("Final assistant message (repr):\n")
    print(final_message)

    print("\nAs plain text:\n")
    print(final_message.content)


## 7. Recap: Why This is a "Complex" but Clear Use Case

You have now seen, end-to-end, how to build a **multi-tool, real-data assistant**:

1. **Simulated MCP server** defines high-level tools:
   - `get_stock_snapshot` (Yahoo Finance via yfinance),
   - `get_company_news` (Tavily search).
2. **MCP client** converts those tools into OpenAI `tools` format.
3. **First model call**:
   - Receives user query + tool list.
   - Chooses which tools to call (with arguments) using `tool_calls`.
4. **Client executes tools**:
   - Calls local functions (simulating MCP `call_tool`).
   - Produces `role="tool"` messages with compact summaries.
5. **Second model call**:
   - Sees tool outputs.
   - Synthesizes a final answer that mixes live data, news, and reasoning.

This is exactly the pattern you can generalize to:

- multiple asset classes,
- more tools (e.g., risk calculators, scenario simulators),
- or even a full agentic system where the model loops through tool calls.

From a **teaching** perspective, this notebook combines:

- Real APIs (Yahoo Finance + Tavily),
- Real OpenAI tool-calling,
- and the conceptual framing of **MCP tools** as capabilities the model can discover and use.
